In [7]:
bp
#backprojection. confidence map

array([[ 38,   3,   0, ...,  24, 255,  82],
       [ 10,  10,   1, ...,  24, 255, 255],
       [  3,   2,   2, ...,  24,  24,  24],
       ...,
       [  9,   9,   9, ...,  10,  16,  16],
       [  9,   9,   9, ...,  16,  10,  10],
       [  9,   9,   9, ...,  16,  10,   8]], dtype=uint8)



```
cv2.meanShift(probImage, window, criteria) -> retval, window
초기에는 WINDOW부터 시작해

```

어떻게 backprojection의 confidence map을 이용해, meanshift를 사용할까?

초기 시작 window의 중점에서 시작하여, 윈도우 내의 confidence들의 무게중심을 계산해 이동한다.

무게중심은 가우시안 분포로 계산된다.

In [19]:
#영상 ROI 지정.

import cv2

mouse_is_pressing = False
start_x,start_y, end_x, end_y = -1,-1,-1,-1
step = 0
track_window = None

#마우스 콜백함수
def mouse_callback(event,x,y,flags,param):
    # 왼쪽 버튼 누를 시 현재 좌표를 사각형 그릴 때
    # 시작 좌표로 저장한다.
    global start_x,start_y,end_x,end_y
    global step,mouse_is_pressing,track_window
    
    if event == cv2.EVENT_LBUTTONDOWN:
        step = 1
        
        mouse_is_pressing = True
        start_x=x
        start_y=y
        
    # 왼쪽 버튼 누른채 마우스 이동시 현재 좌표를 사각형 끝 좌표로
    elif event == cv2.EVENT_MOUSEMOVE:
        if mouse_is_pressing:
            end_x = x
            end_y = y
            step = 2
    #마우스를 떼면 다시 끝좌표로 저장
    elif event == cv2.EVENT_LBUTTONUP:
        mouse_is_pressing = False
        end_x=x
        end_y=y
        step=3

    

In [20]:

#웹캠 초기화
#동영상으로 수정
cap = cv2.VideoCapture('../../data/whale_movie.mp4')
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter('output.mp4', fourcc, 25.0, (int(cap.get(3)),int(cap.get(4))))


if cap.isOpened() == False:
    print("카메라를 열수 없다.")
    exit(1)

#윈도우를 생성하고 마우스 콜백함수 설정
cv2.namedWindow("Color")
cv2.setMouseCallback("Color",mouse_callback)
first=True

while True:
    ret,img_color = cap.read()
    
    if ret==False:
        print("캡처 실패")
        break
    
    if step==1:#처음 클릭시 원을 그림.
        cv2.circle(img_color,(start_x,start_y),10,(0,255,0),-1)
    
    elif step==2:#마우스 이동 중 사각형을 그림.
        cv2.rectangle(img_color,(start_x,start_y),(end_x,end_y),(0,255,0),3)

    elif step==3:## ****손을 뗀 경우 ROI 영역을 얻게됨.
        if start_x > end_x:
            #예외처리
            start_x,end_x = end_x,start_x
            start_y,end_y = end_y,start_y
        #초기 사각형의 위치
        track_window = (start_x,start_y,end_x-start_x,end_y-start_y)
        #x,y,w,h
        
        #HSV 색공간으로 변환하고 ROI영역 지정.
        img_hsv = cv2.cvtColor(img_color,cv2.COLOR_BGR2HSV)
        img_ROI = img_hsv[start_y:end_y,start_x:end_x]
        
        cv2.imshow("ROI",img_ROI)
        
        #ROI의 히스토그램을 계산(명암 히스토그램)
        objectHistogram = cv2.calcHist([img_ROI],[0],None,[180],(0,180))
        #hsv로 변환하고 h채널의 히스토그램을 구한다.
        #(얼굴 검출때와 다르게 색상 채널만)
        
        cv2.normalize(objectHistogram,objectHistogram,alpha=0,beta=255,
                     norm_type=cv2.NORM_MINMAX)
        #모델을 양자화 대신 normalize만 진행.
        #(min-max scaler히스토그램을 0~255 값으로)
        
        step= step+1
    elif step==4:# ROI가 구해진 이후 
        #hsv 색공간으로 변환
        img_hsv = cv2.cvtColor(img_color,cv2.COLOR_BGR2HSV)
        
        #img_hsv에서 model 히스토그램을 갖는 영역을 찾는다.
        #bp는 confidence map이 된다.
        bp = cv2.calcBackProject([img_hsv],[0],objectHistogram,
                                [0,180],1)
        #h 채널 이용
        
        # meanshift를 적용하여, 오브젝트의 새로운 위치를 얻는다.
        term=(cv2.TERM_CRITERIA_EPS|cv2.TERM_CRITERIA_COUNT,10,1)
        #meanshift 종료조건
        ret,track_window = cv2.meanShift(bp,track_window,
                                        term)
        
        #오브젝트 위치에 빨간색 사각형을 그려준다.
        x,y,w,h = track_window
        cv2.rectangle(img_color,(x,y),(x+w,y+h),(0,0,255),2)
    
    out.write(img_color)
    cv2.imshow("Color",img_color)
    if cv2.waitKey(25)>=0:
        break
cap.release()
out.release()
cv2.destroyAllWindows()

캡처 실패
